## The data

As I quickly learned, a data set of sentences in a given language is called a corpus. There are several places open source corpora can be found on the web. I used data from [OPUS](http://opus.lingfil.uu.se/) and the [ANC](http://www.anc.org/). OPUS is actually a repository of parallel corpora which can be used for training machine translation models.

At first I was overly concerned about the quality of the data. I worried over only including sentences with valid English words, for example words that are present in some English dictionary (I used the spell-checking library [PyEnchant](http://pythonhosted.org/pyenchant/tutorial.html)). I worried that since my computational resources, my macbook, are so limited, maybe I should build linguistic annotations into my model, so that instead of having a  single unknown token for all the words not present in my chosen vocabulary I would have for example unknown noun, verb and adjective tokens. To this end, I experimented with the [Stanford POS tagger](http://nlp.stanford.edu/software/tagger.shtml), as exposed through [nltk](http://www.nltk.org/). 

In the end I decided to stop worrying and love the mess. For one thing, it appears to be a point of pride in the papers I've seen that only minimal preprocessing on the corpora is performed. For me, I came to the conclusion that my schemes to filter and supplement the data just made my corpora needlessly small and introduced new POS labeling errors. 

### Specifics

The data from OPUS and ANC is provided as xml files, so it's easy to parse. I tokenized the sentences using nltk and removed all punctuation. I arbitrarily decided to only consider sentences containing between 2 and 32 words. And for my vocabulary, I took the 9998 most common words in my corpora together with two special tokens, <e\> to indicate the absence of a word and <u\> to  indicate an unknown word. 

I ended up with a corpora of 5,696,198 sentences. Of these 5 million will be used for training and the rest will be test data.

### Visualizations 

Miraculously, [Mikolov, Yih, and Zweig](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/rvecs.pdf) show that the simple RNN model that we are playing around with is able to learn a word embedding that contains high-level language structure. To try to get a sense for the type of structure in the corpora that the model has to learn, I built some visualizations with [matplotlib](http://matplotlib.org/)'s [pcolor](http://matplotlib.org/examples/pylab_examples/pcolor_small.html) function. 

The following two charts are provided to introduce the visualization approach. Sentences are shown as rows in a grid with unique words represented by colored cells. The first chart shows eight corpora sentences containing the vocabulary word 'reasonable', the second shows eight sentences containing the word 'fragile', these are, respectively, the 2000-th and 5000-th most common words in the vocabulary. There are 4257 sentences in the corpora that contain the word 'reasonable' and 1071 that contain 'fragile'. In their respective charts, 'reasonable' and 'fragile' are colored red. White cells are unknown words (words not in the vocabularu), black cells are end-of-sentence tokens. 

<img src="img/reasonable_2000_120513_8_sents.png">
<img src="img/fragile_5000_100481_8_sents.png">
 
To get a sense of the scale and complexity of the problem take a look at [500 reasonable sentences](img/reasonable_2000_100481_500_sents.png) and [500 fragile sentences](img/fragile_5000_100481_500_sents.png). Note that for each chart I randomly generated a heat map for the words present in that chart, so heatmaps are not the same between different charts. The colormaps are random because otherwise the only thing we could see in the charts is proximity in the vocabulary. 

Looking at these charts it's difficult to see any structure at all. But of course we know it's there since if you and I were to meet in the street and strike up a conversation about whether or not it's reasonable to install fragile glass sculptures in my three year old's room we would have no difficulty understanding each other. So can a simple visualization show some of this structure?

One thing that appears to work fairly well is to align all the sentences at their occurance of the target word (reasonable or fragile) and sort lexicographically from the left or right on a small neighborhood. Here the sorting is done on the indices of the neighbor words in the vocabulary (their frequency ranking in the corpora) as opposed to on the words and letters themselves. Here's a chart for 'fragile' where we sort from the left on a neighborhood of two words in both directions (only the first 200 sorted sentences are shown):

<img src="img/fragile_5000_2_0_0_100481.png">

Now much more structure is evident. For example, sentences 60 to 80 all contain the phrase 'the most fragile'. I think the following chart is particularly nice. It looks at two-neighbors of 'reasonable' sorted from the right, the 200-th to 500-th sorted sentences are shown. 

<img src="img/reasonable_2000_2_500_1_100481.png">

The phrases 'at a reasonable price', 'at a reasonable cost', and 'at reasonable prices' make up the majority of the structure evident in the above chart. 

### What's next

It's time to see how much of the above structure I can train into an RNN model using my underpowered macbook. The above visualizations at least hint that an appropriate model with enough data could succeed. Let's see what happens! 
